##  Projet Algorithme de texte

SALAS VEGA Dominique - MONET Anaïs

In [1]:
%matplotlib inline

from nltk.corpus import stopwords
from sklearn import metrics
from sklearn import model_selection
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from spacy.lang.en import English
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string

In [2]:
# Suppression de l'affichage des messages d'avertissement
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Lecture du fichier CSV
# Il manquait une apostrophe à la ligne 101
name = ['cuisine','ingredients']
dtypes = {'cuisine':'category', 'ingredients':'object'}
recipe_df = pd.read_csv("train_cuisine.csv", sep=";", names = name , dtype = dtypes, quotechar="'")

# Nettoyage des données : 
# - Suppression des lignes comportant des données manquantes (dropna)
# - Suppression des doublons (drop_duplicates)
recipe_df = recipe_df.drop_duplicates().dropna(how = 'any')

In [4]:
recipe_df.head()

,cuisine,ingredients
0,greek,romaine_lettuce black_olives grape_tomatoes ga...
1,southern_us,plain_flour ground_pepper salt tomatoes ground...
2,filipino,eggs pepper salt mayonaise cooking_oil green_c...
3,indian,water vegetable_oil wheat salt
4,indian,black_pepper shallots cornflour cayenne_pepper...


## Apprentissage
Pour pouvoir créer notre outil de classification, nous allons séparer nos données en deux parties : 
- **`X`** est utilisé pour les données sources utilisées pour l'apprentissage, c'est à dire l'ensemble des traits ici les ingrédients
- **`Y`** est utilisé pour ce que l'on cherche à prédire
- **`train`** données d'apprentissage 
- **`test`** données d'évaluation

In [5]:
X = recipe_df[['ingredients']]
Y = recipe_df.cuisine

In [6]:
X.head()

,ingredients
0,romaine_lettuce black_olives grape_tomatoes ga...
1,plain_flour ground_pepper salt tomatoes ground...
2,eggs pepper salt mayonaise cooking_oil green_c...
3,water vegetable_oil wheat salt
4,black_pepper shallots cornflour cayenne_pepper...


In [7]:
Y.head()

0          greek
1    southern_us
2       filipino
3         indian
4         indian
Name: cuisine, dtype: category
Categories (20, object): [brazilian, british, cajun_creole, chinese, ..., southern_us, spanish, thai, vietnamese]

## Séparation des données 

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

On va transformer le texte dans ingredients en sac de mots pour pouvoir réaliser notre classification

In [9]:
# Classe pour sélectionner des colonnes à partir de leur nom
class SingleColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, data_dict):
        return data_dict[self.key]
    
class MultiColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns].to_dict('records')
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

## Pré-traitement de la colonne ingredients contenant du texte

On va transformer le texte en vecteur. Les ingrédient sont séparés par un underscore. 

In [42]:
# Fonction de tokénisation
def tokenize_ingredients(text):
    return text.split('_')

# Objet CountVectorizer pour la transformation en sac de mots
var_vectorizer = CountVectorizer(tokenizer = tokenize_ingredients, min_df = 0.0001)
    
# Pipeline spécifique
ingredients_pipeline = make_pipeline(
    # Sélection de la colonne "ingredients"
    SingleColumnSelector(key="ingredients"),
    # Transformation sac de mots
    var_vectorizer
)

# Exemple d'application de la pipeline

# Apprentissage du vocabulaire à l'aide de "fit"
ingredients_pipeline.fit(X_train)
# Transformation en sac de mots à l'aide de "transform"
res = ingredients_pipeline.transform(X_test.head())
print("Input")
print(X_test.ingredients.head())
print()
print("Output     Occurences")
print(res)
print()

# Affichage des traits extraits par la pipeline
feat_names = var_vectorizer.get_feature_names()
print(1, feat_names[1])
print(2, feat_names[2])
print(3, feat_names[3])

Input
21441    chipotle_chile garlic_powder onion_powder corn...
24063    sugar cinnamon garlic_cloves onions zucchini g...
34308    sesame_oil wheat_flour water minced_beef soy_s...
9362     freshly_grated_parmesan heavy_cream rigatoni s...
1319     ground_black_pepper dried_oregano dried_thyme ...
Name: ingredients, dtype: object

Output     Occurences
  (0, 390)	1
  (0, 468)	1
  (0, 692)	1
  (0, 1200)	1
  (0, 1251)	1
  (0, 1384)	1
  (0, 1861)	1
  (0, 3061)	1
  (0, 3379)	1
  (0, 4340)	1
  (0, 5059)	1
  (0, 5398)	1
  (0, 6010)	1
  (0, 6017)	1
  (0, 6097)	1
  (0, 7293)	1
  (0, 7374)	1
  (0, 7769)	1
  (1, 146)	1
  (1, 390)	2
  (1, 2041)	1
  (1, 3239)	1
  (1, 3579)	1
  (1, 5529)	1
  (1, 5638)	1
  (2, 379)	1
  (2, 4833)	1
  (2, 7183)	1
  (3, 390)	1
  (3, 611)	1
  (3, 1440)	1
  (3, 2770)	1
  (3, 3011)	1
  (3, 3446)	1
  (3, 4285)	1
  (3, 4723)	1
  (3, 5065)	1
  (3, 5068)	1
  (3, 5367)	1
  (3, 8728)	1
  (3, 8838)	1
  (4, 399)	1
  (4, 3038)	1
  (4, 4983)	1
  (4, 5367)	1
  (4, 5475)	1
  (4, 59

## Lemmatisation


In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/monetanais/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# Fonction de lemmatisation et de découpage en mots
nlp = English()
def split_into_lemmas_spacy(desc) :
    doc = nlp(desc)
    return [w.lemma_ for w in doc]

# Liste des mots outils de NLTK + signes de ponctuation
nltk_stopwords = stopwords.words('english')+list(string.punctuation)
#print(nltk_stopwords)

# Objet TfidfVectorizer
ingred_vectorizer = TfidfVectorizer(tokenizer=split_into_lemmas_spacy, 
                                lowercase=True, 
                                stop_words=nltk_stopwords, 
                                min_df=0.001)

# Pipeline spécifique
ingred_pipeline = make_pipeline(
    SingleColumnSelector(key="ingredients"),
    ingred_vectorizer
)

# Exemple d'application de la pipeline
ingred_pipeline.fit(X_train)
res = ingred_pipeline.transform(X_test.head())
print("Input")
print(X_test.ingredients.iloc[0])
print()
print("Output")
print(res)
print()

# Affichage des traits extraits par la pipeline
feat_names = ingred_vectorizer.get_feature_names()
print(1, feat_names[1])
print(2, feat_names[2])
print(3, feat_names[3])

Input
chipotle_chile garlic_powder onion_powder corn_tortillas lime zucchini sour_cream canola_oil golden_brown_sugar chili_powder coarse_kosher_salt slaw hot_spanish_paprika red_bell_pepper boneless_skinless_chicken_breast_halves

Output
  (0, 1148)	0.2544472607647013
  (0, 994)	0.2254515507931387
  (0, 845)	0.21265073532697001
  (0, 721)	0.28298254076608914
  (0, 641)	0.22787542393162666
  (0, 489)	0.38034042004790697
  (0, 479)	0.22931868812026673
  (0, 276)	0.24897682725126336
  (0, 235)	0.3891945815901242
  (0, 195)	0.34964164222116717
  (0, 187)	0.21009431710267248
  (0, 133)	0.23702686676843018
  (0, 97)	0.2718880522827546
  (1, 1148)	0.2703715110529363
  (1, 1140)	0.3317626904715342
  (1, 1101)	0.15037407636992284
  (1, 1014)	0.15841347693420935
  (1, 845)	0.2259592045285911
  (1, 743)	0.2504292157248876
  (1, 722)	0.14642982495323478
  (1, 583)	0.3815977737129966
  (1, 537)	0.20732949667191103
  (1, 525)	0.32050474557261527
  (1, 477)	0.16023037039122603
  (1, 438)	0.234142549

On va déduire des informations supplémentaires à partir des ingrédients : le nombre d'ingredients.

In [45]:
# Source : https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html

class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, descs):
        return [{'stats_num_sentences': text.count(' ')}
                for text in descs]
    
stats_vectorizer = DictVectorizer()

# Pipeline spécifique
stats_pipeline = make_pipeline(
    SingleColumnSelector(key="ingredients"),
    TextStats(),
    stats_vectorizer
)

# Exemple d'application de la pipeline
stats_pipeline.fit(X_train)
res = stats_pipeline.transform(X_test.head())
print("Input")
print(X_test.ingredients.head())
print()
print("Output       Nombre d'ingrédients")
print(res)
print()

# Affichage des traits extraits par la pipeline
feat_names = stats_vectorizer.get_feature_names()
print(feat_names[0])

Input
21441    chipotle_chile garlic_powder onion_powder corn...
24063    sugar cinnamon garlic_cloves onions zucchini g...
34308    sesame_oil wheat_flour water minced_beef soy_s...
9362     freshly_grated_parmesan heavy_cream rigatoni s...
1319     ground_black_pepper dried_oregano dried_thyme ...
Name: ingredients, dtype: object

Output       Nombre d'ingrédients
  (0, 0)	14.0
  (1, 0)	14.0
  (2, 0)	7.0
  (3, 0)	11.0
  (4, 0)	7.0

stats_num_sentences


In [46]:
# Union des traits
union = FeatureUnion(transformer_list = [
        ("ingreds_feature", ingred_pipeline),
        ("stats_features", stats_pipeline)
    ])

# Chaîne de prétraitement globale, composée de l'union des chaînes
preprocess_pipeline = make_pipeline(
    union
)

# Application de la chaîne à X_train
preprocess_pipeline.fit(X_train)
X_transformed = preprocess_pipeline.transform(X_train)
print(X_transformed.shape)

#Affichage du nombre de traits générés par chacune des chaines 
fnames_stat = stats_pipeline.named_steps['dictvectorizer'].get_feature_names()
fnames_ingred = ingred_pipeline.named_steps['tfidfvectorizer'].get_feature_names()
print('# stats features : ', len(fnames_stat))
print('# ingred features : ', len(fnames_ingred))

(31268, 1151)
# stats features :  1
# ingred features :  1150


In [47]:
fnames_ingred[0:10]

['\r\n',
 '1\\%_low',
 '2\\%_reduced',
 'active_dry_yeast',
 'adobo_sauce',
 'agave_nectar',
 'alfredo_sauce',
 'all_purpose_unbleached_flour',
 'allspice',
 'almond_extract']

In [48]:
# Prétaitement + apprentissage
classifier_pipeline = make_pipeline(
    preprocess_pipeline,
    LogisticRegression()
)
# Apprentissage avec les données d'entraînement
classifier_pipeline.fit(X_train, Y_train)
# Test sur des données issues du jeu de test (uniquement les premières lignes)
predicted = classifier_pipeline.predict(X_test.head(20))
print(X_test.head(20))
print()
print("Predicted :")
print(predicted)
print()
print("Expected : ")
print(Y_test.head(20))

                                             ingredients
21441  chipotle_chile garlic_powder onion_powder corn...
24063  sugar cinnamon garlic_cloves onions zucchini g...
34308  sesame_oil wheat_flour water minced_beef soy_s...
9362   freshly_grated_parmesan heavy_cream rigatoni s...
1319   ground_black_pepper dried_oregano dried_thyme ...
27153  powdered_sugar unsalted_butter all-purpose_flo...
16044  crushed_tomatoes diced_tomatoes creole_seasoni...
32228  spinach part-skim_mozzarella_cheese garlic_clo...
16254  light_mayonnaise feta_cheese_crumbles pepper s...
35674  soy_sauce ginger scallions vegetable_oil rice_...
8397   kosher_salt ginger oil chicken_wings szechwan_...
13497  chicken_broth water_chestnuts black_moss oyste...
5023   fennel_seeds pork_belly granulated_sugar groun...
9048   salt bacon ground_beef tomato_juice rice papri...
3025   chicken_breasts oil water salt nian_gao white_...
10120  sugar all-purpose_flour ice_water gala_apples ...
16139  water salsa taco_sauce l

In [49]:
all_predictions = classifier_pipeline.predict(X_test)
print(all_predictions)

['mexican' 'moroccan' 'chinese' ... 'mexican' 'spanish' 'jamaican']


## Calcul de l'exactitude

In [50]:
# Source: https://stackoverflow.com/questions/54875846/how-to-print-labels-and-column-names-for-confusion-matrix
print('accuracy', accuracy_score(Y_test, all_predictions))
labels = np.unique(Y_test)
cm =  confusion_matrix(Y_test, all_predictions, labels=labels)
confusion_df = pd.DataFrame(cm, index=labels, columns=labels)
print('confusion matrix\n', confusion_df)
print('(row=expected, col=predicted)')

accuracy 0.7409823484267076
confusion matrix
               brazilian  british  cajun_creole  chinese  filipino  french  \
brazilian            25        1             2        0         3       2   
british               1       38             1        2         2      32   
cajun_creole          0        2           215        3         0      15   
chinese               2        1             1      436         5       3   
filipino              1        1             0       26        68       2   
french                0        3             5        3         0     295   
greek                 0        0             0        1         0       8   
indian                0        0             0        1         1       6   
irish                 0       10             1        0         0      14   
italian               0        6             2        2         0      50   
jamaican              1        4             1        3         0       4   
japanese              0       

## Partie2

In [101]:
df = pd.read_json("./recipes_raw/recipes_raw_nosource_epi.json", orient = 'index')
df

,ingredients,instructions,picture_link,title
..1fLeH8MEEaHhoimrhhh39zHkJiywe,"[1 cup diced peeled clementines, 1/2 cup chopp...","Mix clementines, red onion, cilantro, 1 1/2 ta...",None,Ancho-Rubbed Steaks with Clemetine-Red Onion S...
..1q2exyLrX7e57eayExiARxQNYn0he,"[2 tablespoons olive oil, 12 ounces boneless b...",1. Put the oil in a large pot or Dutch oven ov...,None,Wheat Berries with Braised Beef and Parsnips
..6JSfE/ggpOYvDiXumBuMxy3VAQFky,"[1 1/3 cups sugar, 2/3 cup water, 2 small nave...",Bring sugar and water to a boil in a 1- to 1 1...,None,Spiced Cranberry and Orange Relish
..MuX2dFe7TK6uQtzF2iHGS2zbGwuv2,"[1 bunch radishes (about 6), 1 tablespoon unsa...",Halve radishes lengthwise and thinly slice the...,None,Sauteed Radishes with Chives
..iu/KtHe7kYRcW2WbGoqwTZyxfgVoa,"[6 large artichokes, 3 lemons, quartered lengt...",For artichokes: Cut top half off 1 artichoke; ...,None,Artichoke and Celery Root Salad with Mustard D...
...,...,...,...,...
zzCrbwS9owo8t94bccFhFx.ox3WiKY6,[1 red jalapeño or Fresno chile with some seed...,"Whisk chile, oil, lime juice, soy sauce, brown...",tu5vHR7v4x4.mgosSc6qXPC/UBcn.SW,Asian Chicken and Cabbage Salad
zzOlMeSbL.ekciudjNN5TCYDQygVQK2,"[1/2 cup fresh tangerine juice, 1 jalapeño, 1 ...",Simmer tangerine juice in a small saucepan unt...,.0ik7nzCOBrvpbGfKY4pcC6BdFPEgn6,Avocado and Tangerine Salad with Jalapeño Vina...
zzSlSaw5M.pu2vVf/zFk1I/u.WKUIie,"[1/2 cup old-fashioned oats, 1/4 cup all-purpo...",For crumble: Whisk all ingredients except butt...,MLmd/akffxJefAgp.6USDcq/DVAzRg6,Cherry-Bourbon Pie
zzXgScyonT7iqoBfD0iKB1Pwbj/odOK,"[1 tablespoon olive oil, 8 skin-on, bone-in ch...",Preheat oven to 425°F. Heat oil in a large ove...,peljRrZje3UfKOLu9E1toE1OTp5DVJe,Pan-Roasted Chicken with Harissa Chickpeas


In [109]:
df['ingredients'] = df['ingredients'].astype(str)
df['ingredients'] = df['ingredients'].str.replace('[', '').str.replace(']', '')
ing = df[['ingredients']]
ing

,ingredients
..1fLeH8MEEaHhoimrhhh39zHkJiywe,"'1 cup diced peeled clementines', '1/2 cup cho..."
..1q2exyLrX7e57eayExiARxQNYn0he,"'2 tablespoons olive oil', '12 ounces boneless..."
..6JSfE/ggpOYvDiXumBuMxy3VAQFky,"'1 1/3 cups sugar', '2/3 cup water', '2 small ..."
..MuX2dFe7TK6uQtzF2iHGS2zbGwuv2,"'1 bunch radishes (about 6)', '1 tablespoon un..."
..iu/KtHe7kYRcW2WbGoqwTZyxfgVoa,"'6 large artichokes', '3 lemons, quartered len..."
...,...
zzCrbwS9owo8t94bccFhFx.ox3WiKY6,'1 red jalapeño or Fresno chile with some seed...
zzOlMeSbL.ekciudjNN5TCYDQygVQK2,"'1/2 cup fresh tangerine juice', '1 jalapeño',..."
zzSlSaw5M.pu2vVf/zFk1I/u.WKUIie,"'1/2 cup old-fashioned oats', '1/4 cup all-pur..."
zzXgScyonT7iqoBfD0iKB1Pwbj/odOK,"'1 tablespoon olive oil', '8 skin-on, bone-in ..."


In [110]:
predictions = classifier_pipeline.predict(ing)

In [111]:
print(predictions)

['southern_us' 'jamaican' 'chinese' ... 'cajun_creole' 'moroccan'
 'southern_us']


In [138]:
import json 

def removeLevel(d):
    if type(d) != type({}):
        return d
    final = []
    for k, v in d.items():
        removed = {}
        if type(v) != type({}):
            continue
        for kk, vv in v.items():
            removed[kk] = vv
        final.append(removed)
    return final

path = './recipes_raw/recipes_raw_nosource_epi.json'
with open(path) as json_data:
    data = json.load(json_data)

# Si on ne renommait pas la first Key notre foncton removeLevel ne fonctionne pas
id = "id"
data = dict(zip([id+str(k) for k in range(df.shape[0])], data.values()))
    
i = 0
for key,value in data.items():
    print(" Key :", key)
    value.update({'origine' : predictions[i]})
    value.update({'id' : str(i)})
    i +=1 

data = removeLevel(data)


with open(path, 'w') as file:
    json.dump(data, file)



 Key : id0
 Key : id1
 Key : id2
 Key : id3
 Key : id4
 Key : id5
 Key : id6
 Key : id7
 Key : id8
 Key : id9
 Key : id10
 Key : id11
 Key : id12
 Key : id13
 Key : id14
 Key : id15
 Key : id16
 Key : id17
 Key : id18
 Key : id19
 Key : id20
 Key : id21
 Key : id22
 Key : id23
 Key : id24
 Key : id25
 Key : id26
 Key : id27
 Key : id28
 Key : id29
 Key : id30
 Key : id31
 Key : id32
 Key : id33
 Key : id34
 Key : id35
 Key : id36
 Key : id37
 Key : id38
 Key : id39
 Key : id40
 Key : id41
 Key : id42
 Key : id43
 Key : id44
 Key : id45
 Key : id46
 Key : id47
 Key : id48
 Key : id49
 Key : id50
 Key : id51
 Key : id52
 Key : id53
 Key : id54
 Key : id55
 Key : id56
 Key : id57
 Key : id58
 Key : id59
 Key : id60
 Key : id61
 Key : id62
 Key : id63
 Key : id64
 Key : id65
 Key : id66
 Key : id67
 Key : id68
 Key : id69
 Key : id70
 Key : id71
 Key : id72
 Key : id73
 Key : id74
 Key : id75
 Key : id76
 Key : id77
 Key : id78
 Key : id79
 Key : id80
 Key : id81
 Key : id82
 Key : id83
 K

 Key : id757
 Key : id758
 Key : id759
 Key : id760
 Key : id761
 Key : id762
 Key : id763
 Key : id764
 Key : id765
 Key : id766
 Key : id767
 Key : id768
 Key : id769
 Key : id770
 Key : id771
 Key : id772
 Key : id773
 Key : id774
 Key : id775
 Key : id776
 Key : id777
 Key : id778
 Key : id779
 Key : id780
 Key : id781
 Key : id782
 Key : id783
 Key : id784
 Key : id785
 Key : id786
 Key : id787
 Key : id788
 Key : id789
 Key : id790
 Key : id791
 Key : id792
 Key : id793
 Key : id794
 Key : id795
 Key : id796
 Key : id797
 Key : id798
 Key : id799
 Key : id800
 Key : id801
 Key : id802
 Key : id803
 Key : id804
 Key : id805
 Key : id806
 Key : id807
 Key : id808
 Key : id809
 Key : id810
 Key : id811
 Key : id812
 Key : id813
 Key : id814
 Key : id815
 Key : id816
 Key : id817
 Key : id818
 Key : id819
 Key : id820
 Key : id821
 Key : id822
 Key : id823
 Key : id824
 Key : id825
 Key : id826
 Key : id827
 Key : id828
 Key : id829
 Key : id830
 Key : id831
 Key : id832
 Key : id833

 Key : id1507
 Key : id1508
 Key : id1509
 Key : id1510
 Key : id1511
 Key : id1512
 Key : id1513
 Key : id1514
 Key : id1515
 Key : id1516
 Key : id1517
 Key : id1518
 Key : id1519
 Key : id1520
 Key : id1521
 Key : id1522
 Key : id1523
 Key : id1524
 Key : id1525
 Key : id1526
 Key : id1527
 Key : id1528
 Key : id1529
 Key : id1530
 Key : id1531
 Key : id1532
 Key : id1533
 Key : id1534
 Key : id1535
 Key : id1536
 Key : id1537
 Key : id1538
 Key : id1539
 Key : id1540
 Key : id1541
 Key : id1542
 Key : id1543
 Key : id1544
 Key : id1545
 Key : id1546
 Key : id1547
 Key : id1548
 Key : id1549
 Key : id1550
 Key : id1551
 Key : id1552
 Key : id1553
 Key : id1554
 Key : id1555
 Key : id1556
 Key : id1557
 Key : id1558
 Key : id1559
 Key : id1560
 Key : id1561
 Key : id1562
 Key : id1563
 Key : id1564
 Key : id1565
 Key : id1566
 Key : id1567
 Key : id1568
 Key : id1569
 Key : id1570
 Key : id1571
 Key : id1572
 Key : id1573
 Key : id1574
 Key : id1575
 Key : id1576
 Key : id1577
 Key :

 Key : id2257
 Key : id2258
 Key : id2259
 Key : id2260
 Key : id2261
 Key : id2262
 Key : id2263
 Key : id2264
 Key : id2265
 Key : id2266
 Key : id2267
 Key : id2268
 Key : id2269
 Key : id2270
 Key : id2271
 Key : id2272
 Key : id2273
 Key : id2274
 Key : id2275
 Key : id2276
 Key : id2277
 Key : id2278
 Key : id2279
 Key : id2280
 Key : id2281
 Key : id2282
 Key : id2283
 Key : id2284
 Key : id2285
 Key : id2286
 Key : id2287
 Key : id2288
 Key : id2289
 Key : id2290
 Key : id2291
 Key : id2292
 Key : id2293
 Key : id2294
 Key : id2295
 Key : id2296
 Key : id2297
 Key : id2298
 Key : id2299
 Key : id2300
 Key : id2301
 Key : id2302
 Key : id2303
 Key : id2304
 Key : id2305
 Key : id2306
 Key : id2307
 Key : id2308
 Key : id2309
 Key : id2310
 Key : id2311
 Key : id2312
 Key : id2313
 Key : id2314
 Key : id2315
 Key : id2316
 Key : id2317
 Key : id2318
 Key : id2319
 Key : id2320
 Key : id2321
 Key : id2322
 Key : id2323
 Key : id2324
 Key : id2325
 Key : id2326
 Key : id2327
 Key :

 Key : id3007
 Key : id3008
 Key : id3009
 Key : id3010
 Key : id3011
 Key : id3012
 Key : id3013
 Key : id3014
 Key : id3015
 Key : id3016
 Key : id3017
 Key : id3018
 Key : id3019
 Key : id3020
 Key : id3021
 Key : id3022
 Key : id3023
 Key : id3024
 Key : id3025
 Key : id3026
 Key : id3027
 Key : id3028
 Key : id3029
 Key : id3030
 Key : id3031
 Key : id3032
 Key : id3033
 Key : id3034
 Key : id3035
 Key : id3036
 Key : id3037
 Key : id3038
 Key : id3039
 Key : id3040
 Key : id3041
 Key : id3042
 Key : id3043
 Key : id3044
 Key : id3045
 Key : id3046
 Key : id3047
 Key : id3048
 Key : id3049
 Key : id3050
 Key : id3051
 Key : id3052
 Key : id3053
 Key : id3054
 Key : id3055
 Key : id3056
 Key : id3057
 Key : id3058
 Key : id3059
 Key : id3060
 Key : id3061
 Key : id3062
 Key : id3063
 Key : id3064
 Key : id3065
 Key : id3066
 Key : id3067
 Key : id3068
 Key : id3069
 Key : id3070
 Key : id3071
 Key : id3072
 Key : id3073
 Key : id3074
 Key : id3075
 Key : id3076
 Key : id3077
 Key :

...

 Key : id22121
 Key : id22122
 Key : id22123
 Key : id22124
 Key : id22125
 Key : id22126
 Key : id22127
 Key : id22128
 Key : id22129
 Key : id22130
 Key : id22131
 Key : id22132
 Key : id22133
 Key : id22134
 Key : id22135
 Key : id22136
 Key : id22137
 Key : id22138
 Key : id22139
 Key : id22140
 Key : id22141
 Key : id22142
 Key : id22143
 Key : id22144
 Key : id22145
 Key : id22146
 Key : id22147
 Key : id22148
 Key : id22149
 Key : id22150
 Key : id22151
 Key : id22152
 Key : id22153
 Key : id22154
 Key : id22155
 Key : id22156
 Key : id22157
 Key : id22158
 Key : id22159
 Key : id22160
 Key : id22161
 Key : id22162
 Key : id22163
 Key : id22164
 Key : id22165
 Key : id22166
 Key : id22167
 Key : id22168
 Key : id22169
 Key : id22170
 Key : id22171
 Key : id22172
 Key : id22173
 Key : id22174
 Key : id22175
 Key : id22176
 Key : id22177
 Key : id22178
 Key : id22179
 Key : id22180
 Key : id22181
 Key : id22182
 Key : id22183
 Key : id22184
 Key : id22185
 Key : id22186
 Key : id2

 Key : id22948
 Key : id22949
 Key : id22950
 Key : id22951
 Key : id22952
 Key : id22953
 Key : id22954
 Key : id22955
 Key : id22956
 Key : id22957
 Key : id22958
 Key : id22959
 Key : id22960
 Key : id22961
 Key : id22962
 Key : id22963
 Key : id22964
 Key : id22965
 Key : id22966
 Key : id22967
 Key : id22968
 Key : id22969
 Key : id22970
 Key : id22971
 Key : id22972
 Key : id22973
 Key : id22974
 Key : id22975
 Key : id22976
 Key : id22977
 Key : id22978
 Key : id22979
 Key : id22980
 Key : id22981
 Key : id22982
 Key : id22983
 Key : id22984
 Key : id22985
 Key : id22986
 Key : id22987
 Key : id22988
 Key : id22989
 Key : id22990
 Key : id22991
 Key : id22992
 Key : id22993
 Key : id22994
 Key : id22995
 Key : id22996
 Key : id22997
 Key : id22998
 Key : id22999
 Key : id23000
 Key : id23001
 Key : id23002
 Key : id23003
 Key : id23004
 Key : id23005
 Key : id23006
 Key : id23007
 Key : id23008
 Key : id23009
 Key : id23010
 Key : id23011
 Key : id23012
 Key : id23013
 Key : id2

 Key : id23615
 Key : id23616
 Key : id23617
 Key : id23618
 Key : id23619
 Key : id23620
 Key : id23621
 Key : id23622
 Key : id23623
 Key : id23624
 Key : id23625
 Key : id23626
 Key : id23627
 Key : id23628
 Key : id23629
 Key : id23630
 Key : id23631
 Key : id23632
 Key : id23633
 Key : id23634
 Key : id23635
 Key : id23636
 Key : id23637
 Key : id23638
 Key : id23639
 Key : id23640
 Key : id23641
 Key : id23642
 Key : id23643
 Key : id23644
 Key : id23645
 Key : id23646
 Key : id23647
 Key : id23648
 Key : id23649
 Key : id23650
 Key : id23651
 Key : id23652
 Key : id23653
 Key : id23654
 Key : id23655
 Key : id23656
 Key : id23657
 Key : id23658
 Key : id23659
 Key : id23660
 Key : id23661
 Key : id23662
 Key : id23663
 Key : id23664
 Key : id23665
 Key : id23666
 Key : id23667
 Key : id23668
 Key : id23669
 Key : id23670
 Key : id23671
 Key : id23672
 Key : id23673
 Key : id23674
 Key : id23675
 Key : id23676
 Key : id23677
 Key : id23678
 Key : id23679
 Key : id23680
 Key : id2

 Key : id24384
 Key : id24385
 Key : id24386
 Key : id24387
 Key : id24388
 Key : id24389
 Key : id24390
 Key : id24391
 Key : id24392
 Key : id24393
 Key : id24394
 Key : id24395
 Key : id24396
 Key : id24397
 Key : id24398
 Key : id24399
 Key : id24400
 Key : id24401
 Key : id24402
 Key : id24403
 Key : id24404
 Key : id24405
 Key : id24406
 Key : id24407
 Key : id24408
 Key : id24409
 Key : id24410
 Key : id24411
 Key : id24412
 Key : id24413
 Key : id24414
 Key : id24415
 Key : id24416
 Key : id24417
 Key : id24418
 Key : id24419
 Key : id24420
 Key : id24421
 Key : id24422
 Key : id24423
 Key : id24424
 Key : id24425
 Key : id24426
 Key : id24427
 Key : id24428
 Key : id24429
 Key : id24430
 Key : id24431
 Key : id24432
 Key : id24433
 Key : id24434
 Key : id24435
 Key : id24436
 Key : id24437
 Key : id24438
 Key : id24439
 Key : id24440
 Key : id24441
 Key : id24442
 Key : id24443
 Key : id24444
 Key : id24445
 Key : id24446
 Key : id24447
 Key : id24448
 Key : id24449
 Key : id2

 Key : id25054
 Key : id25055
 Key : id25056
 Key : id25057
 Key : id25058
 Key : id25059
 Key : id25060
 Key : id25061
 Key : id25062
 Key : id25063
 Key : id25064
 Key : id25065
 Key : id25066
 Key : id25067
 Key : id25068
 Key : id25069
 Key : id25070
 Key : id25071
 Key : id25072
 Key : id25073
 Key : id25074
 Key : id25075
 Key : id25076
 Key : id25077
 Key : id25078
 Key : id25079
 Key : id25080
 Key : id25081
 Key : id25082
 Key : id25083
 Key : id25084
 Key : id25085
 Key : id25086
 Key : id25087
 Key : id25088
 Key : id25089
 Key : id25090
 Key : id25091
 Key : id25092
 Key : id25093
 Key : id25094
 Key : id25095
 Key : id25096
 Key : id25097
 Key : id25098
 Key : id25099
 Key : id25100
 Key : id25101
 Key : id25102
 Key : id25103
 Key : id25104
 Key : id25105
 Key : id25106
 Key : id25107
 Key : id25108
 Key : id25109
 Key : id25110
 Key : id25111
 Key : id25112
 Key : id25113
 Key : id25114
 Key : id25115
 Key : id25116
 Key : id25117
 Key : id25118
 Key : id25119
 Key : id2

In [139]:
data #vérification du format

[{'ingredients': ['12 egg whites',
   '12 egg yolks',
   '1 1/2 cups sugar',
   '3/4 cup rye whiskey',
   '12 egg whites',
   '3/4 cup brandy',
   '1/2 cup rum',
   '1 to 2 cups heavy cream, lightly whipped',
   'Garnish: ground nutmeg'],
  'picture_link': None,
  'instructions': 'Beat the egg whites until stiff, gradually adding in 3/4 cup sugar. Set aside. Beat the egg yolks until they are thick and pale and add the other 3/4 cup sugar and stir in rye whiskey. Blend well. Fold the egg white mixture into the yolk mixture and add the brandy and the rum. Beat the mixture well. To serve, fold the lightly whipped heavy cream into the eggnog. (If a thinner mixture is desired, add the heavy cream unwhipped.) Sprinkle the top of the eggnog with the nutmeg to taste.\nBeat the egg whites until stiff, gradually adding in 3/4 cup sugar. Set aside. Beat the egg yolks until they are thick and pale and add the other 3/4 cup sugar and stir in rye whiskey. Blend well. Fold the egg white mixture into 